### Import libraries:

In [1]:
import requests
import time
import re
from bs4 import BeautifulSoup
from itertools import chain
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

### Set a user agent:

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15"}

### Pick artists to scrape:

In [48]:
print("Please choose your artists. If artist's name is composed of 2 or more words, please put a hyphen (-) in between")
artists = []
inputs = artists
while inputs != '0':
    inputs = input('input artists (press 0 when done): ')
    artists.append(inputs)

Please choose your artists. If artist's name is composed of 2 or more words, please put a hyphen (-) in between
input artists (press 0 when done): 2pac
input artists (press 0 when done): nas
input artists (press 0 when done): mos-def
input artists (press 0 when done): notorious-big
input artists (press 0 when done): 0


In [50]:
artists = artists[:-1]

Check what's in the list:

In [51]:
artists

['2pac', 'nas', 'mos-def', 'notorious-big']

### Get lyrics-page to scrape from for each artist in list:

In [6]:
lyrics_links = []
for artist in artists:
    url = 'https://www.metrolyrics.com/' + artist.lower() + '-lyrics.html'
    http_request = requests.get(url, headers=headers)
    pattern = 'http://www.metrolyrics.com/[^\s]+lyrics-' + artist + '\.html'
    lyrics_links.append(re.findall(pattern, http_request.text))

In [42]:
len(lyrics_links[0]), len(lyrics_links[1]), len(lyrics_links[2]), len(lyrics_links[3])

(79, 79, 79, 78)

In [8]:
lyrics_links[2]

['http://www.metrolyrics.com/ms-fat-booty-lyrics-mos-def.html',
 'http://www.metrolyrics.com/six-days-lyrics-mos-def.html',
 'http://www.metrolyrics.com/respiration-lyrics-mos-def.html',
 'http://www.metrolyrics.com/mathematics-lyrics-mos-def.html',
 'http://www.metrolyrics.com/ms-fat-booty-lyrics-mos-def.html',
 'http://www.metrolyrics.com/six-days-lyrics-mos-def.html',
 'http://www.metrolyrics.com/respiration-lyrics-mos-def.html',
 'http://www.metrolyrics.com/mathematics-lyrics-mos-def.html',
 'http://www.metrolyrics.com/oh-no-lyrics-mos-def.html',
 'http://www.metrolyrics.com/travellin-man-lyrics-mos-def.html',
 'http://www.metrolyrics.com/umi-says-lyrics-mos-def.html',
 'http://www.metrolyrics.com/a-soldiers-dream-lyrics-mos-def.html',
 'http://www.metrolyrics.com/quiet-dog-bite-hard-lyrics-mos-def.html',
 'http://www.metrolyrics.com/new-world-water-lyrics-mos-def.html',
 'http://www.metrolyrics.com/perfect-timing-lyrics-mos-def.html',
 'http://www.metrolyrics.com/fear-not-of-man-l

CAVEAT: some songpages might be empty, i.e. missing lyrics. Only non-empty songs will be appended to the songlist below.

### Scrape the lyrics for each artist song:

In [9]:
songs = []
artist_list = []
# for artist in artists:
for element in lyrics_links:
    for link in element:
        time.sleep(1)                                             # pauses 1 sec
        song_page = requests.get(link, headers=headers)           # requesting each url
        song_soup = BeautifulSoup(song_page.text)                 # soups url-text
        verse = song_soup.body.find_all(class_="verse")           # finds all verses
        songtext = ' '.join([x.text for x in verse])              # loops through verses and extracts text only
        if songtext != '':                                        # checks for empty strings (missing lyrics) before appending
            song_artist = link.split('/')[-1].split('.')[0].split('lyrics-')[-1] #splits the url to label each artist of a (non-empty) song
            artist_list.append(song_artist)                       # creates a list of artist labels
            songs.append(songtext)                                # having all songtexts in a list
            #song_name = link.split('/')[-1].split('.')[0]          # splits the url to label each song
    #        with open(artist + '_lyrics.txt', 'a') as f:                # saves to txt with song-names and empty lines
    #            f.write('\n\n' + songtext + '\n')

In [10]:
print(artist_list) # this list serves as labels for the index of the matrix/dataframe

['2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'na

check the number of scraped songs per artist (must be counter-balanced eventually):

In [11]:
number_of_songs = []
for artist in artists:
    number_of_songs.append(artist_list.count(artist))

In [47]:
number_of_songs, sum(number_of_songs)

([79, 76, 70, 74], 299)

In [13]:
len(songs) #number of songs in list 

299

In [14]:
print(songs[-1])

Yo, C-Gutta where you at?
D Rockafella
Caesar Leo Degenero
Bled C, Money SL B.I.G. is making this cream
Bitches always say what the hell does that mean?
B is for the bitches, who can't understand
How one fly nigga became a man I is for the way it goes in and out
One by one I knocks 'em out
G is for the way the game goes in the gutter Other MC's, man, they ain't sayin' nuthin'
Rockin' on to the break of dawn
Make the bitch give the pussy
Get the mouth, I'm gone What? It ain't no more to it


creating artist labels:

In [15]:
labels = artist_list

# labels = [artists[0]] * number_of_songs[0] + [artists[1]] * number_of_songs[1] + [artists[2]] * number_of_songs[2]

In [16]:
len(labels) #checking for number of labels (should equal number of scraped songs in total):

299

check for correct labels:

In [17]:
print(labels)

['2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', '2pac', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'nas', 'na

In [18]:
corpus = songs # copying the songs into another list to clean them

In [19]:
len(corpus)

299

### Clean the corpus (with RegEx):

In [20]:
cleaned_corpus = []
for song in corpus:
    song = song.lower()                                     # changes all to lower case
    song = re.sub(r"[-()\"#/@;:<>{}=~|.?,\*\!']", "", song) # deletes special characters
    song = re.sub(r"[\[].*?[\]]", "", song)                 # deletes additional info in square brackets
    song = re.sub("\d+", "", song)                          # deletes digits
    cleaned_corpus.append(song)

### Transforming the corpus into a feature matrix (TfidfVectorizer):

In [21]:
tf = TfidfVectorizer()
tf.fit(cleaned_corpus)
tf_vec_corpus = tf.transform(cleaned_corpus)
df = pd.DataFrame(tf_vec_corpus.todense().round(2), index=labels, columns=tf.get_feature_names())

In [22]:
df

,aaaaaahhhhhh,aaahhh,aalike,aarow,aaw,ab,abandoned,abbaccas,abboys,abdomen,...,zhane,zinc,zion,zip,zone,zoo,zoot,zovarax,zulu,zyaire
2pac,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2pac,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2pac,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2pac,0.0,0.0,0.0,0.0,0.29,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2pac,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
notorious-big,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
notorious-big,0.0,0.1,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
notorious-big,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
notorious-big,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Defining X and y:

In [23]:
X = df
y = df.index

#### Check if the data is balanced!

In [24]:
df.index.value_counts(normalize=True) #proportion of songs per artist

2pac             0.264214
nas              0.254181
notorious-big    0.247492
mos-def          0.234114
dtype: float64

In [60]:
df.index.value_counts()

2pac             79
nas              76
notorious-big    74
mos-def          70
dtype: int64

In [25]:
df.isna().values.sum() # check for missing (NaN) values

0

### Dealing with class imbalance (by Undersampling):

RandomUndersampling randomly deletes samples from the majority class(es) to fit number of minority class

In [26]:
from imblearn.under_sampling import RandomUnderSampler

In [27]:
rus = RandomUnderSampler()

In [28]:
X_rus, y_rus = rus.fit_resample(X, y)

In [29]:
X.shape, X_rus.shape

((299, 12403), (280, 12403))

## Model: Multinomial Naive Bayes

In [30]:
m = MultinomialNB()

In [31]:
m.fit(X_rus, y_rus)

MultinomialNB()

In [52]:
new_input = [input("Please enter the verse you want to know the artist of: ")]

Please enter the verse you want to know the artist of: they wanna burn your molecules until you let go ions treat beef like they let go lions so don't eye 'em


In [53]:
new_input

["they wanna burn your molecules until you let go ions treat beef like they let go lions so don't eye 'em"]

In [54]:
vector_input = tf.transform(new_input)

In [55]:
y_pred = m.predict(vector_input)

In [56]:
y_pred

array(['notorious-big'], dtype='<U13')

In [57]:
probability = m.predict_proba(vector_input)

In [58]:
probability

array([[0.26882552, 0.21593944, 0.21458239, 0.30065265]])

In [59]:
print(f"The artist who wrote this verse is most likely {y_pred[0]} with a probability of {round(probability.max(), 2)*100}%")

The artist who wrote this verse is most likely notorious-big with a probability of 30.0%
